In [22]:
import re
import pandas as pd
import numpy as np
#from sklearn.preprocessing import LabelEncoder
#from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk

In [23]:
train_sentences = pd.read_csv("../data/dataset_mr/Trainset_complete.csv", sep=';') 
test_sentences = pd.read_csv("../data/dataset_mr/Testset.csv", sep=';') 

In [24]:
train_sentences.head(10)

,Unnamed: 0,SentenceId,PhraseId,Phrase,Sentiment
0,878,879,19873,A fifty car pileup of cliches .,-1
1,501,502,11621,The film has the thrown-together feel of a sum...,-1
2,70,71,1918,But it has an ambition to say something about ...,-1
3,1065,1066,23510,They presume their audience wo n't sit still f...,-1
4,556,557,12955,Make Chan 's action sequences boring .,-1
5,941,942,21052,Sheridan,0
6,489,490,11422,"Romantic , riveting and handsomely animated .",1
7,812,813,18531,An ambitious and moving but bleak film .,0
8,600,601,13949,Limps along on a squirm-inducing fish-out-of-w...,-1
9,257,258,6503,Genuinely unnerving .,1


In [25]:
s = 0.0
for i in train_sentences['Phrase']:
    word_list = i.split()
    s = s + len(word_list)
print("Average length of each review : ",s/data.shape[0])
pos = 0
neg = 0
for i in range(train_sentences.shape[0]):
    if train_sentences.iloc[i]['Sentiment'] == 1:
        pos = pos + 1
    elif train_sentences.iloc[i]['Sentiment'] == -1:
        neg = neg + 1
    
neu = train_sentences.shape[0]-pos-neg
print("Percentage of reviews with positive sentiment is "+str(pos/train_sentences.shape[0]*100)+"%")
print("Percentage of reviews with negative sentiment is "+str(neg/train_sentences.shape[0]*100)+"%")
print("Percentage of reviews with neutrag sentiment is "+str(neu/train_sentences.shape[0]*100)+"%")

Average length of each review :  18.783382789317507
Percentage of reviews with positive sentiment is 33.33333333333333%
Percentage of reviews with negative sentiment is 33.33333333333333%
Percentage of reviews with neutrag sentiment is 33.33333333333333%


#### Before being fed into the LSTM model, the data needs to be padded and tokenized:
#### Tokenizing: Keras’ inbuilt tokenizer API has fit the dataset, which splits the sentences into words and creates a dictionary of all unique words found and their uniquely assigned integers. Each sentence is converted into an array of integers representing all the individual words present in it.
#### Sequence Padding: The array representing each sentence in the dataset is filled with zeroes to the left to make the size of the array ten and bring all collections to the same length.

In [26]:
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = ''
embedding_dim = 100
max_length = 200 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)
# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

An embedding layer of dimension 100 converts each word in the sentence into a fixed-length dense vector of size 100. The input dimension is set as the vocabulary size, and the output dimension is 100. Each word in the input will hence get represented by a vector of size 100.
A bidirectional LSTM layer of 64 units.
A dense (fully connected) layer of 24 units with relu activation.
A dense layer of 1 unit and sigmoid activation outputs the probability of the review is positive, i.e. if the label is 1

In [27]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 100)          300000    
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               84480     
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 24)                3096      
                                                                 
 dense_3 (Dense)             (None, 1)                 25        
                                                                 
Total params: 387601 (1.48 MB)
Trainable params: 387601 (1.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
train_labels = train_sentences['Sentiment'].values
test_labels = test_sentences['Sentiment'].values
train_labels

array([-1, -1, -1, ..., -1, -1,  0], dtype=int64)

In [32]:
num_epochs = 5
history = model.fit(train_padded, train_labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/5
1/1 [==============================] - 0s 239ms/step - loss: 0.1774 - accuracy: 0.0000e+00 - val_loss: 0.0198 - val_accuracy: 0.0000e+00
Epoch 2/5
1/1 [==============================] - 0s 160ms/step - loss: 0.0113 - accuracy: 0.0000e+00 - val_loss: -0.1927 - val_accuracy: 0.0000e+00
Epoch 3/5
1/1 [==============================] - 0s 143ms/step - loss: -0.2024 - accuracy: 0.0000e+00 - val_loss: -0.4720 - val_accuracy: 0.0000e+00
Epoch 4/5
1/1 [==============================] - 0s 156ms/step - loss: -0.4830 - accuracy: 0.0000e+00 - val_loss: -0.8390 - val_accuracy: 0.0000e+00
Epoch 5/5
1/1 [==============================] - 0s 151ms/step - loss: -0.8511 - accuracy: 0.0000e+00 - val_loss: -1.3003 - val_accuracy: 0.0000e+00


In [34]:
prediction = model.predict(test_padded)
# Get labels based on probability 1 if p>= 0.5 else 0
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    elif (i < 0.5 and i >= -0.5):
         pred_labels.append(0)
    else:
        pred_labels.append(-1)
print("Accuracy of prediction on test set : ", accuracy_score(test_labels,pred_labels))

1/1 [==============================] - 0s 67ms/step


ValueError: Found input variables with inconsistent numbers of samples: [1377, 5]